In [40]:
## adding dependencies
import json
import requests
from pprint import pprint
import pandas as pd
from sqlalchemy import create_engine
import pymongo
from pandas.io.json import json_normalize
import hashlib
import time


In [44]:

m = hashlib.md5()
ts = str(time.time())
ts_byte = bytes(ts, 'utf-8');
m.update(ts_byte)
m.update(b"b999c2120532176b4609989cdd0b56242036cc97")
m.update(b"b2aeb1c91ad82792e4583eb08509f87a")
         

hasht = m.hexdigest()
hasht


'd9a041dcaa3b1707e986120cfb9b0437'

In [45]:
##base might be https://gateway.marvel.com:443
base_url = "https://gateway.marvel.com"
api_key = "b2aeb1c91ad82792e4583eb08509f87a"
query = "/v1/public/events" +"?"

query_url = base_url + query +"ts=" + ts+ "&apikey=" + api_key + "&hash=" + hasht
query_url

'https://gateway.marvel.com/v1/public/events?ts=1560451289.349657&apikey=b2aeb1c91ad82792e4583eb08509f87a&hash=d9a041dcaa3b1707e986120cfb9b0437'

In [46]:
data = requests.get(query_url).json()
data

{'code': 200,
 'status': 'Ok',
 'copyright': '© 2019 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2019 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2019 MARVEL</a>',
 'etag': 'b537c5c5869458fc880939aaf7e533ae36d060e9',
 'data': {'offset': 0,
  'limit': 20,
  'total': 75,
  'count': 20,
  'results': [{'id': 116,
    'title': 'Acts of Vengeance!',
    'description': 'Loki sets about convincing the super-villains of Earth to attack heroes other than those they normally fight in an attempt to destroy the Avengers to absolve his guilt over inadvertently creating the team in the first place.',
    'resourceURI': 'http://gateway.marvel.com/v1/public/events/116',
    'urls': [{'type': 'detail',
      'url': 'http://marvel.com/comics/events/116/acts_of_vengeance?utm_campaign=apiRef&utm_source=b2aeb1c91ad82792e4583eb08509f87a'},
     {'type': 'wiki',
      'url': 'http://marvel.com/universe/Acts_of_Vengeance!?utm_campaign=apiRef&utm_source=b2ae

In [ ]:
# Items of interest are the Title, description, start, end,  and all characters

In [37]:
eventTitle = []
eventDesc =[]
eventStart = []
eventEnd = []
eventCharacters =[]

for x in data["data"]["results"]:
    eTitle = x["title"]
    eventTitle.append(eTitle)
    eDesc = x["description"]
    eventDesc.append(eDesc)
    eStart = x["start"]
    eventStart.append(eStart)
    eEnd = x["end"]
    eventEnd.append(eEnd)
    eCharacters = x["characters"]
    eventCharacters.append(eCharacters)
    
    
    
Eventdf = pd.DataFrame()
Eventdf["Name"] = eventTitle
Eventdf["Desc"] = eventDesc
Eventdf["Start_Date"] = eventStart
Eventdf["End_Date"] = eventEnd
Eventdf["Characters"] = eventCharacters

Eventdf.head()


,Name,Desc,Start_Date,End_Date,Characters
0,Acts of Vengeance!,Loki sets about convincing the super-villains ...,1989-12-10 00:00:00,2008-01-04 00:00:00,"{'available': 102, 'collectionURI': 'http://ga..."
1,Age of Apocalypse,"In a twisted version of the world they knew, t...",1995-03-01 00:00:00,1996-06-01 00:00:00,"{'available': 91, 'collectionURI': 'http://gat..."
2,Age of Ultron,In a 10-issue event written by Brian Michael B...,2013-03-06 00:00:00,2013-06-19 00:00:00,"{'available': 13, 'collectionURI': 'http://gat..."
3,Age of X,In a strange reality where the X-Men never cam...,2011-01-26 00:00:00,2011-05-01 00:00:00,"{'available': 45, 'collectionURI': 'http://gat..."
4,All-New All-Different Marvel,"This fall, prepare for an all-new, all-differe...",None,None,"{'available': 0, 'collectionURI': 'http://gate..."


In [38]:
#Connecting the MongoDatabase
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.Marvel_events

In [39]:
#Creating the database Marvel_events
collection = db.Event_Details
df = Eventdf
records_ = df.to_dict(orient = 'records')
result = db.Event_Details.insert_many(records_)
result


